In [16]:
import pandas as pd
import bokeh
import astropy
from astropy import units as u
from astropy import constants as c
from astropy.coordinates import SkyCoord
import json
from math import pi
import numpy as np

In [17]:
exos=pd.read_csv('exoplanets.csv')
exos[['RightAscension', 'Declination']].head()
print np.min(exos['DistFromSunParsec']), np.max(exos['DistFromSunParsec'])

1.3417 8500.0


In [48]:
def get_values(row):
    
    name=str(obj[1]['PlanetIdentifier'])
    jupiter_mass=possiblefail(obj,'PlanetaryMassJpt')
    earth_mass=tryingtoderive(jupiter_mass, earthmass)
    axis_au=possiblefail(obj,'SemiMajorAxisAU')
    disc_method=str(obj[1]['DiscoveryMethod'])
    disc_year=possiblefail(obj,'DiscoveryYear')
    ra=possiblefail(obj,'RightAscension')
    dec=possiblefail(obj,'Declination')
    distance=possiblefail(obj, 'DistFromSunParsec')
    host_mass=possiblefail(obj, 'HostStarMassSlrMass')
    host_temp=possiblefail(obj, 'HostStarTempK')
    host_lum=tryingtoderive(host_temp, lum)
    hostmag=tryingtoderive(host_lum, abs_mag)
    try:
        apparentmagnitude=app_mag(hostmag, distance)
    except ValueError:
        apparentmagnitude=5.0
    if apparentmagnitude==np.nan or apparentmagnitude=="None":
        apparentmagnitude=5.0
    return name, jupiter_mass, earth_mass, axis_au, disc_method, disc_year, ra, dec, distance, host_mass, apparentmagnitude
        
def possiblefail(obj, valuename):
    if pd.isnull(obj[1][valuename])==False:
        try:
            possiblevalue=obj[1][valuename]
        except ValueError:
            possiblevalue='None'
        return possiblevalue
    else:
        return np.nan

def tryingtoderive(value, func):
    try:
        new_val=func(value)
    except ValueError:
        new_val='None'
    return new_val

def earthmass(jup_mass):
    return 317.8*jup_mass

def lum(temp):
    return 4.*pi*c.R_sun.to(u.m).value**2*c.sigma_sb.value*temp**4

def abs_mag(lum):
    return -2.5*np.log10(lum/c.L_sun.value)

def app_mag(absmag, dist):
    if pd.isnull(dist)==True:
        return 5.
    else:
        return 5.*np.log10(dist)+absmag-5.

In [50]:
out=open('exos_radians.json','w')
#out.write('#name, apparentmagnitude, ra, dec, jupiter_mass, earth_mass, axis_au, disc_method, disc_year, distance, host_mass\n')
ss_planets=['Mercury','Venus','Earth','Mars','Jupiter','Saturn','Uranus', 'Neptune','Pluto']
exos_rad=[]
for obj in exos.iterrows():
    cont=True
    name, jupiter_mass, earth_mass, axis_au, disc_method, disc_year, ra, dec, distance, host_mass, apparentmagnitude=get_values(obj)
    print name, apparentmagnitude
    for planet in ss_planets:
        if name==planet:
            cont=False
    if ra=='None' or dec=='None':
        cont=False
    if cont==True:
        if apparentmagnitude=="nan" or pd.isnull(apparentmagnitude)==True or apparentmagnitude>5.:
            apparentmagnitude=5.0
        elif apparentmagnitude<-0.5:
            apparentmagnitude=-0.5
        print apparentmagnitude
        ra=ra.replace(' ',':')
        dec=dec.replace(' ',':')
        #print ra, dec
        s = SkyCoord(ra+' '+dec, unit=(u.hourangle, u.deg))
        l=[name, apparentmagnitude, s.ra.radian, s.dec.radian, jupiter_mass, earth_mass, axis_au, disc_method, disc_year, distance, host_mass]
        exos_rad.append(l)
out.write(json.dumps({'exos':exos_rad}))
out.close()
# Output values: name, apparentmagnitude, ra, dec, jupiter_mass, earth_mass, axis_au, disc_method, disc_year, distance, host_mass

KOI-1843.03 5.0
5.0
KOI-1843.01 5.0
5.0
KOI-1843.02 5.0
5.0
Kepler-9 b 9.06650192875
5.0
Kepler-9 c 9.06650192875
5.0
Kepler-9 d 9.06650192875
5.0
GJ 160.2 b 4.41699297805
4.41699297805
Kepler-566 b 5.0
5.0
WASP-124 b 7.98384455619
5.0
HD 240210 b 7.06398547144
5.0
OGLE-05-390L b 15.8402133605
5.0
Kepler-1343 b 5.0
5.0
Kepler-1011 b 5.0
5.0
WASP-82 b 6.00166183127
5.0
Gliese 1214 b 3.31444017777
3.31444017777
Kepler-153 b 9.27502393638
5.0
Kepler-153 c 9.27502393638
5.0
HD 4203 b 4.51414282355
4.51414282355
HD 4203 c 4.51414282355
4.51414282355
HD 179079 b 4.0623188145
4.0623188145
Kepler-1368 b 5.0
5.0
HD 187123 b 3.39319728406
3.39319728406
HD 187123 c 3.39319728406
3.39319728406
Kepler-1111 b 5.0
5.0
Kepler-1289 b 5.0
5.0
Kepler-431 b 5.0
5.0
Kepler-431 c 5.0
5.0
Kepler-431 d 5.0
5.0
Kepler-131 b 6.84507690049
5.0
Kepler-131 c 6.84507690049
5.0
Kepler-548 b 5.0
5.0
HD 114729 A b nan
5.0
CoRoT-24 b 9.56366308353
5.0
CoRoT-24 c 9.56366308353
5.0
Kepler-1105 b 5.0
5.0
Kepler-211 c 8.10